# Lambert Conformal Conic

## Definitions

See https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection

Let us have the following input parameters:

| Symbol | Parameter |
| ------ | --------- |
| X_0    | Reference Longitude |
| Y_0    | Reference Latitude  |
| R      | Radius of the Earth |
| Y_1    | First standard parallel |
| Y_2    | Second standard parallel |

Derived numbers:

| Symbol | Formula |
| ------ | ------- |
| n      | ln( cosY_1 / cosY_2 ) / ln( tan(pi/4 + Y_2/2) / tan(pi/4 + Y_1/2) ) |
| F      | 1/n( cosY_1 tan^n(pi/4 + Y_1/2) )                                   |
| P_0    | R * F * cot^n (pi/4 + Y_0/2                                         |

Calculations for converting longitude, latitude (X, Y) to (U, V): 

| Symbol | Formula |
| ------ | ------- |
| P      | R * F * cot^n (pi/4 + Y/2)  |
| U      | P sin(n * (X - X_0) )       |
| V      | P_0 - P cos(n * (X - X_0) ) |

## Straightforward Code

In [12]:
import numpy as np

def lambert_convert(x, y, R, x0, y0, y1, y2, degrees=True):
    if degrees:
        x0 = (np.pi/180)*x0
        y0 = (np.pi/180)*y0
        y1 = (np.pi/180)*y1
        y2 = (np.pi/180)*y2
        
        x = (np.pi/180)*x
        y = (np.pi/180)*y
    
    n  = (np.log(np.cos(y1)) - np.log(np.cos(y2))) / (np.log(np.tan(np.pi/4 + y2/2) / np.tan(np.pi/4 + y1/2)))
    F  = (np.cos(y1)*(np.tan(np.pi/4 + y1/2)**n))/n
    p0 = R*F*(np.tan(np.pi/4 + y0/2)**(-n))
    
    p = R*F*(np.tan(np.pi/4 + y/2)**(-n))
    
    u =      p*np.sin(n*(x - x0))
    v = p0 - p*np.cos(n*(x - x0))
    
    return u,v

In [15]:
lambert_convert(0.3, 51.5, 6300, 0, 50, 45, 60)

(20.36295105396931, 163.6598131597484)

## Optimise for np-arrays

# Albers Equal-Area Conic

## Definitions

Let us define:

| Symbol | Definition |
| ------ | ---------- |
| R      | Radius of the Earth |
| x0     | Reference longitude |
| y0     | Reference latitude  |
| y1     | First standard parallel |
| y2     | Second standard parallel |
| x      | Input longitude |
| y      | Input latitude  |

Formulas:

| Symbol | Formula |
| ------ | ------- |
| n      | 1/2 ( sin(y1) + sin(y2) ) |
| theta  | n * (x - x0 ) |
| C      | cos(y1)^2 + 2 * n * sin(y1) |
| P      | (R/n) * sqrt( C - 2 * n * sin(y) ) |
| P0     | (R/n) * sqrt( C - 2 * n * sin(y0) ) |
| U      | P sin(theta) |
| V      | P0 - P cos(theta) |

## Straightforward Code

In [1]:
import numpy as np

def albers_convert(x, y, R, x0, y0, y1, y2, degrees=True):
    if degrees:
        x0 = (np.pi/180)*x0
        y0 = (np.pi/180)*y0
        y1 = (np.pi/180)*y1
        y2 = (np.pi/180)*y2
        
        x = (np.pi/180)*x
        y = (np.pi/180)*y
    
    n = (np.sin(y1) + np.sin(y2)) / 2
    c = np.cos(y1)**2 + 2*n*np.sin(y1)
    p  = (R/n)*np.sqrt(c - 2*n*np.sin(y))
    p0 = (R/n)*np.sqrt(c - 2*n*np.sin(y0))
    
    t = n*(x - x0)
    u =      p*np.sin(t)
    v = p0 - p*np.cos(t)
    
    return u,v

In [2]:
albers_convert(0.3, 51.5, 6300, 0, 50, 45, 60)

(20.367109716856532, 166.25650317185955)

## Optimised for np-arrays

In [1]:
import numpy as np

from bokeh.plotting import figure, show
from bokeh.palettes import Viridis256, Inferno256
from bokeh.layouts import row, column
from bokeh.io import output_notebook    # In-notebook visualisation
output_notebook()

Loading BokehJS ...

In [2]:
class Albers:
    '''A class to carry out Albers equal-area conical projections'''
    
    def __init__(self, x0, y0, y1, y2, degrees=True):
        '''Parameters
        
        degrees : bool
            If True, the other parameters are given in degrees; otherwise radians
        x0, y0 : floats
            Reference longitude, latitude (coordinates of origin)
        y1, y2 : floats
            Standard parallels
        '''
        self.degrees = degrees
        k = np.pi/180 if degrees else 1
        self.x0      = x0*k
        self.y0      = y0*k
        self.y1      = y1*k
        self.y2      = y2*k
        
        self.R = 6374 # Radius of Earth in kilometres
        
        self.n  = (np.sin(self.y1) + np.sin(self.y2)) / 2
        self.c  = np.cos(self.y1)**2 + 2*self.n*np.sin(self.y1)
        self.p0 = (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(self.y0))
        
        return None
    
    
    def scale(self, array):
        return (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(array))
    
    
    def convert(self, coords, degrees=True):
        
        if degrees:
            coords = coords*(np.pi/180)
            
        p = self.scale(coords[:,1])
        t = self.n*(coords[:,0] - self.x0)
        u = p*np.sin(t)
        v = self.p0 - p*np.cos(t)
        
        return np.c_[u,v]
    

# Optimising the UK range

In [3]:
import numpy as np

from bokeh.plotting import figure, show
from bokeh.palettes import Viridis256, Inferno256
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column
from bokeh.io import output_notebook    # In-notebook visualisation
output_notebook()

Loading BokehJS ...

In [121]:
class Albers:
    '''A class to carry out Albers equal-area conical projections'''
    
    def __init__(self, x0, y0, y1, y2, degrees=True):
        '''Parameters
        
        degrees : bool
            If True, the other parameters are given in degrees; otherwise radians
        x0, y0 : floats
            Reference longitude, latitude (coordinates of origin)
        y1, y2 : floats
            Standard parallels
        '''
        k = np.pi/180 if degrees else 1
        self.x0      = x0*k
        self.y0      = y0*k
        self.y1      = y1*k
        self.y2      = y2*k
        
        self.R = 6371 # Radius of Earth in kilometres
        
        self.n  = (np.sin(self.y1) + np.sin(self.y2)) / 2
        self.c  = np.cos(self.y1)**2 + 2*self.n*np.sin(self.y1)
        self.p0 = (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(self.y0))
        
        self.North = self.convert(np.array([[x0,90]]))
        self.origin = np.array([[x0, y0]])
        self.degrees = degrees
        
        return None
    
    
    def scale(self, array):
        return (self.R/self.n)*np.sqrt(self.c - 2*self.n*np.sin(array))
    
    
    def convert(self, coords, degrees=True):
        k = np.pi/180 if degrees else 1
        array = coords*k
            
        p = self.scale(array[:,1])
        t = self.n*(array[:,0] - self.x0)
        u = p*np.sin(t)
        v = self.p0 - p*np.cos(t)
        
        return np.c_[u,v]
    
    
    def measures(self, uv_coords):
        
        distances  = np.linalg.norm(uv_coords, axis=1)
        
        norths     = self.North - uv_coords
        norths    /= np.linalg.norm(norths, axis=1)[:,None]
        north_devs = np.abs(np.arctan2(norths[:,1], -norths[:,0]) - np.pi/2)
        
        bearings   = np.arctan2(-uv_coords[:,0], -uv_coords[:,1])
        
        return distances, north_devs, bearings
    
    
    def deviation(self, coords, degrees=True):
        k = 1 if degrees else np.pi/180
        l = 1 if self.degrees else 180/np.pi
        origin = self.origin*k*l
        
        true_distances  = latlong_distances(  coords, origin, degrees=degrees)
        true_north_devs = latlong_north_devs( coords, origin, degrees=degrees)
        true_bearings   = latlong_bearings(   coords, origin, degrees=degrees)
        
        uv_coords = self.convert(coords, degrees=degrees)
        map_distances, map_north_devs, map_bearings = self.measures(uv_coords)
        
        d_dist = np.abs(true_distances[:,0]  - map_distances)
        d_ndev = np.abs(true_north_devs[:,0] - map_north_devs)
        d_bear = np.abs(true_bearings[:,0]   - map_bearings)
        
#         print(np.max(true_distances), np.max(true_north_devs), np.max(true_bearings))
#         print(np.max(map_distances), np.max(map_north_devs), np.max(map_bearings))
#         print('\n')
#         print(np.min(true_distances), np.min(true_north_devs), np.min(true_bearings))
#         print(np.min(map_distances), np.min(map_north_devs), np.min(map_bearings))
#         print('\n')
#         print(np.max(d_dist), np.max(d_ndev), np.max(d_bear))
#         print(np.min(d_dist), np.min(d_ndev), np.min(d_bear))
        
        return d_dist, d_ndev, d_bear
    
    
    def average_deviation(self, coords, degrees=True):
        
        dist, ndev, bear = self.deviation(coords, degrees=degrees)
        m_dist  = np.mean(dist)
        m_ndev  = np.mean(ndev)
        m_bear  = np.mean(bear)
        m_total = np.linalg.norm(np.array([[m_dist, m_ndev, m_bear]]))
        
        return m_dist, m_ndev, m_bear, m_total

In [94]:
def latlong_to_xyz(coords, degrees=True, radius=6371):
    '''Return 3-vectors for position on the Earth.
    Input ndarray of shape (n, 2). Columns : 0=longitude, 1=latitude
    '''
    k = np.pi/180 if degrees else 1
    array = coords*k
    
    return np.c_[
        radius*np.cos(array[:,1])*np.cos(array[:,0]),
        radius*np.cos(array[:,1])*np.sin(array[:,0]),
        radius*np.sin(array[:,1])
    ]
    
    
def latlong_to_norths(coords, degrees=True):
    '''Return unit 3-vectors for the north direction on a sphere.
    Input ndarray of shape (n, 2). Columns : 0=longitude, 1=latitude
    '''
    k = np.pi/180 if degrees else 1
    array = coords*k
    return np.c_[
        np.cos(np.pi/2 + array[:,1])*np.cos(array[:,0]),
        np.cos(np.pi/2 + array[:,1])*np.sin(array[:,0]),
        np.sin(np.pi/2 + array[:,1])
    ]
    
    
def latlong_to_easts(coords, degrees=True):
    '''Return unit 3-vectors for the north direction on a sphere.
    Input ndarray of shape (n, 2). Columns : 0=longitude, 1=latitude
    '''
    k = np.pi/180 if degrees else 1
    array = coords*k
    return np.c_[
        np.cos(np.pi/2 + array[:,0]),
        np.sin(np.pi/2 + array[:,0]),
        0
    ]
    
    
def latlong_distances(A, B, degrees=True, radius=6371):
    '''Return spherical distance matrix on Earth.
    Input ndarrays of shape (a, 2), (b, 2). Columns : 0=longitude, 1=latitude
    Output ndarray of shape (a, b).
    
    distance = acos(sin(lat1)*sin(lat2)+cos(lat1)*cos(lat2)*cos(lon2-lon1))*6371
    '''
    k = np.pi/180 if degrees else 1
    arr_a = (A*k)[:,None,:]
    arr_b = (B*k)[None,:,:]
    
    sines    = np.sin(arr_a[:,:,1])*np.sin(arr_b[:,:,1])
    cosines  = np.cos(arr_a[:,:,1])*np.cos(arr_b[:,:,1])
    long_cos = np.cos(arr_b[:,:,0] - arr_a[:,:,0])
    argument = sines + cosines*long_cos
    return np.arccos(argument)*radius


def latlong_north_devs(A, B, degrees=True):
    '''Return matrix of angles between north vectors on a sphere.
    Input ndarrays of shape (a, 2), (b, 2). Columns : 0=longitude, 1=latitude
    Output ndarray of shape (a, b).
    
    '''    
    a_north  = latlong_to_norths(A, degrees=degrees)
    b_north  = latlong_to_norths(B, degrees=degrees)
    b_east   = latlong_to_easts( B, degrees=degrees)
    
    arg_1 = np.linalg.norm(np.cross(a_north[:,None,:], b_east[None,:,:]), axis=2)
    arg_2 = np.dot(a_north, b_east.T)
    
    return np.abs(np.arctan2(arg_1, arg_2) - np.pi/2)
    #return -np.arcsin(np.linalg.norm(np.cross(a_north[:,None,:], b_east[None,:,:]), axis=2))


def latlong_bearings(A, B, degrees=True):
    '''Return bearings matrix on a sphere.
    Input ndarrays of shape (a, 2), (b, 2). Columns : 0=longitude, 1=latitude
    Output ndarray of shape (a, b).
    
    bearing = arctan2(
        sin(lon2-lon1)*cos(lat2),
        cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(lon2-lon1)
    )
    '''
    k = np.pi/180 if degrees else 1
    arr_a = (A*k)[:,None,:]
    arr_b = (B*k)[None,:,:]
    
    arg_1  = np.sin(arr_b[:,:,0] - arr_a[:,:,0])*np.cos(arr_b[:,:,1])
    arg_2a = np.cos(arr_a[:,:,1])*np.sin(arr_b[:,:,1])
    arg_2b = np.sin(arr_a[:,:,1])*np.cos(arr_b[:,:,1])*np.cos(arr_b[:,:,0]-arr_a[:,:,0])
    
    return np.arctan2(arg_1, arg_2a - arg_2b)

In [137]:
R = 6371

N = 2500
UK_sample_set  = np.random.random(size=(N,2))
UK_sample_set *= np.array([[13, 11]])
UK_sample_set += np.array([[-2, 49]])
bool_1 = UK_sample_set[:,1] <= 68 - 1.4*UK_sample_set[:,0]
bool_2 = UK_sample_set[:,1] <= 56 + 1.4*UK_sample_set[:,0]
UK_sample_set = UK_sample_set[bool_1 & bool_2]

London = np.array([[0, 51.5]])

sample_distances  = latlong_distances(  UK_sample_set, London )
sample_north_devs = latlong_north_devs( UK_sample_set, London )
sample_bearings   = latlong_bearings(   UK_sample_set, London )

In [123]:
d_min, n_min, b_min = np.min(np.c_[sample_distances, sample_north_devs, sample_bearings], axis=0)
d_max, n_max, b_max = np.max(np.c_[sample_distances, sample_north_devs, sample_bearings], axis=0)

uk_data = ColumnDataSource(data=dict(
    xs   = -UK_sample_set[:,0].copy(),
    ys   = UK_sample_set[:,1].copy(),
    dists = sample_distances[:,0].copy(),
    Ndevs = sample_north_devs[:,0].copy(),
    bears = sample_bearings[:,0].copy()
))

dist = figure(width=250, height=250, match_aspect=True)
devs = figure(width=250, height=250, match_aspect=True)
bear = figure(width=250, height=250, match_aspect=True)

dist.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('dists', Viridis256, d_min, d_max))
devs.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('Ndevs', Viridis256, n_min, n_max))
bear.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('bears', Viridis256, b_min, b_max))

show(row(dist, devs, bear))

In [124]:
A = Albers(London[0,0], London[0,1], 40, 60)

In [125]:
UK_converted = A.convert(UK_sample_set)
map_dist, map_ndev, map_bear = A.measures(UK_converted)

ds_min, ns_min, bs_min = np.min(np.c_[sample_distances, sample_north_devs, sample_bearings], axis=0)
ds_max, ns_max, bs_max = np.max(np.c_[sample_distances, sample_north_devs, sample_bearings], axis=0)
dm_min, nm_min, bm_min = np.min(np.c_[map_dist, map_ndev, map_bear], axis=0)
dm_max, nm_max, bm_max = np.max(np.c_[map_dist, map_ndev, map_bear], axis=0)

uk_data = ColumnDataSource(data=dict(
    xs   = -UK_sample_set[:,0].copy(),
    ys   = UK_sample_set[:,1].copy(),
    dists = sample_distances[:,0].copy(),
    ndevs = sample_north_devs[:,0].copy(),
    bears = sample_bearings[:,0].copy(),
    distm = map_dist,
    ndevm = map_ndev,
    bearm = map_bear
))

f_dist = figure(width=250, height=250, match_aspect=True)
f_ndev = figure(width=250, height=250, match_aspect=True)
f_bear = figure(width=250, height=250, match_aspect=True)
g_dist = figure(width=250, height=250, match_aspect=True)
g_ndev = figure(width=250, height=250, match_aspect=True)
g_bear = figure(width=250, height=250, match_aspect=True)

f_dist.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('dists', Viridis256, dm_min, dm_max))
f_ndev.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('ndevs', Viridis256, ns_min, nm_max))
f_bear.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('bears', Viridis256, bm_min, bm_max))
g_dist.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('distm', Viridis256, dm_min, dm_max))
g_ndev.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('ndevm', Viridis256, ns_min, nm_max))
g_bear.scatter('xs', 'ys', source=uk_data, size=4, color=linear_cmap('bearm', Viridis256, bm_min, bm_max))

show(column(
    row(f_dist, f_ndev, f_bear),
    row(g_dist, g_ndev, g_bear)
))

In [126]:
UK_converted           = A.convert(UK_sample_set)
d_dist, d_ndev, d_bear = A.deviation(UK_sample_set)

In [127]:
d_min, n_min, b_min = np.min(np.c_[d_dist, d_ndev, d_bear], axis=0)
d_max, n_max, b_max = np.max(np.c_[d_dist, d_ndev, d_bear], axis=0)

d_data = ColumnDataSource(data=dict(
    xs   = -UK_converted[:,0].copy(),
    ys   =  UK_converted[:,1].copy(),
    dists = d_dist.copy(),
    Ndevs = d_ndev.copy(),
    bears = d_bear.copy()
))

dist = figure(width=250, height=250, match_aspect=True)
devs = figure(width=250, height=250, match_aspect=True)
bear = figure(width=250, height=250, match_aspect=True)

dist.scatter('xs', 'ys', source=d_data, size=4, color=linear_cmap('dists', Viridis256, d_min, d_max))
devs.scatter('xs', 'ys', source=d_data, size=4, color=linear_cmap('Ndevs', Viridis256, n_min, n_max))
bear.scatter('xs', 'ys', source=d_data, size=4, color=linear_cmap('bears', Viridis256, b_min, b_max))

show(row(dist, devs, bear))

In [179]:
def warp_run(y1, y2, var, steps):

    xs = np.linspace(y1-var, y1+var, steps)
    ys = np.linspace(y2-var, y2+var, steps)
    
    dist = np.zeros((steps, steps))
    ndev = np.zeros((steps, steps))
    bear = np.zeros((steps, steps))
    mean = np.zeros((steps, steps))
    
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            array = UK_sample_set.copy()
            A = Albers(London[0,0], London[0,1], x, y, degrees=True)
            dist[i,j], ndev[i,j], bear[i,j], mean[i,j] = A.average_deviation(array)
            
    f_dist = figure(height=300, width=300, match_aspect=True)
    f_ndev = figure(height=300, width=300, match_aspect=True)
    f_bear = figure(height=300, width=300, match_aspect=True)
    f_mean = figure(height=300, width=300, match_aspect=True)

    dist_lvl = np.linspace(np.min(dist), 0.5, 40)
    dist_con = f_dist.contour(xs, ys, dist.T, dist_lvl, fill_color=Inferno256)
    dist_bar = dist_con.construct_color_bar()
    f_dist.add_layout(dist_bar, "right")

    ndev_lvl = np.linspace(np.min(ndev), np.max(ndev), 20)
    ndev_con = f_ndev.contour(xs, ys, ndev.T, ndev_lvl, fill_color=Inferno256)
    ndev_bar = ndev_con.construct_color_bar()
    f_ndev.add_layout(ndev_bar, "right")

    bear_lvl = np.linspace(np.min(bear), np.max(bear), 20)
    bear_con = f_bear.contour(xs, ys, bear.T, bear_lvl, fill_color=Inferno256)
    bear_bar = bear_con.construct_color_bar()
    f_bear.add_layout(bear_bar, "right")

    mean_lvl = np.linspace(np.min(mean), np.max(mean), 20)
    mean_con = f_mean.contour(xs, ys, mean.T, mean_lvl, fill_color=Inferno256)
    mean_bar = mean_con.construct_color_bar()
    f_mean.add_layout(mean_bar, "right")
    
    show(column(
        row(f_dist, f_ndev),
        row(f_bear, f_mean)
    ))
    return None
# test_albers_warp_uk(40, 50)

In [180]:
warp_run(51, 57, 2, 60)